In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# 1. Data analysis

In [21]:
data = pd.read_csv('labeled.csv')
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [22]:
data.shape

(14412, 2)

In [23]:
data['toxic'] = data['toxic'].apply(int)
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1
1,"Хохлы, это отдушина затюканого россиянина, мол...",1
2,Собаке - собачья смерть\n,1
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1


In [24]:
data.value_counts('toxic')

toxic
0    9586
1    4826
dtype: int64

In [25]:
for true_value in data[data['toxic'] == 1]['comment'].head():
    print(true_value)

Верблюдов-то за что? Дебилы, бл...

Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.

Собаке - собачья смерть

Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?

тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)



In [26]:
for false_value in data[data['toxic'] == 0]['comment'].head():
    print(false_value)

В шапке были ссылки на инфу по текущему фильму марвел. Эти ссылки были заменены на фразу Репортим брипидора, игнорируем его посты. Если этого недостаточно, чтобы понять, что модератор абсолютный неадекват, и его нужно лишить полномочий, тогда эта борда пробивает абсолютное дно по неадекватности.

Почитайте посты у этого автора,может найдете что нибудь полезное. Надеюсь помог) https: pikabu.ru story obyichnyie budni dezsluzhbyi 4932098

Про графику было обидно) я так то проходил все серии гта со второй части по пятую, кроме гта 4. И мне не мешала графика ни в одной из частей. На компе у меня было куча видеокарт. Начиная с 32мб RIVA TNT и заканчивая 2Гб 560Ti на которой я спокойно играю который год в танки, гта5, ведьмака3 купил на распродаже и начал проходить. Да, не на ультрах. С пониженными текстурами. И не мешает. Я не понимаю дрочева на графике, требовать графику уровня плойки 4 минимум. Мне надо чтобы глаза не резало, только и всего. По поводу управления, мне не хватает переходника

In [27]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size = 500, random_state = 0)
test_data.shape

(500, 2)

In [28]:
test_data['toxic'].value_counts()

0    346
1    154
Name: toxic, dtype: int64

In [29]:
train_data['toxic'].value_counts()

0    9240
1    4672
Name: toxic, dtype: int64

# 2. Data preprocessing

In [30]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import string

nltk.download('punkt')
nltk.download('stopwords')

sentence_example = data.loc[0, 'comment'] # 'Верблюдов-то за что? Дебилы, бл...\n'

tokens = word_tokenize(sentence_example, language = 'russian') # ['Верблюдов-то', 'за', 'что', '?', 'Дебилы', ',', 'бл', '...']
tokens_without_punctuation = [i for i in tokens if i not in string.punctuation] # ['Верблюдов-то', 'за', 'что', 'Дебилы', 'бл', '...']

russian_stop_words = stopwords.words('russian') # ['и', 'в', 'во', 'не', ...]
tokens_without_russian_stop_words_and_punctuation = [i for i in tokens_without_punctuation if i not in russian_stop_words] # ['Верблюдов-то', 'Дебилы', 'бл', '...']

snowball = SnowballStemmer(language = 'russian')
stemmed_tokens = [snowball.stem(i) for i in tokens_without_russian_stop_words_and_punctuation] # ['верблюдов-т', 'дебил', 'бл', '...']

print(f"Исходный текст: {sentence_example}")
print(f"Обработанный текст: {stemmed_tokens}")

Исходный текст: Верблюдов-то за что? Дебилы, бл...

Обработанный текст: ['верблюдов-т', 'дебил', 'бл', '...']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
# Стоп слова и Стемминг
russian_stop_words = stopwords.words('russian')
snowball = SnowballStemmer(language = 'russian')

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language = 'russian')
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

tokenize_sentence(sentence_example)

['верблюдов-т', 'дебил', 'бл', '...']

<font size="5">***TF-IDF: Учитываем важность слова***</font>

TF-IDF is a feature vectorization method widely used in text analysis. TF-IDF helps to reflect the importance of the word both in the document and throughout the corpus. The corpus is the totality of all documents. TF-IDF consists of two components: Term Frequency (TF) and Inverse Document Frequency (IDF).

![](https://associatedpresscapstone.github.io/img/tfidf.png)

TF is calculated as the ratio of the occurrence of words to the total number of words in the document. The IDF part is counted for each word in the dictionary, not in the document. N is the number of documents in the corpus, and DF is the number of documents in which the word occurs. If the word occurs in all documents, then IDF = 0.

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x, remove_stop_words = True))
features = vectorizer.fit_transform(train_data['comment'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# 3. Model training

In [33]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state = 0)
model.fit(features, train_data['toxic']) # Implementation of the model using logistic regression

LogisticRegression(random_state=0)

In [34]:
model.predict(features[0])

array([0])

In [35]:
train_data['comment'].iloc[0]

'Кровопускание уже вернулось:\n'

<font size="5">***Pipeline***</font>

![](https://spark-school.ru/wp-content/uploads/2021/01/ml-pipelinemodel2.png)

In [36]:
from sklearn.pipeline import Pipeline

model_pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x, remove_stop_words = True))),
    ('model', LogisticRegression(random_state = 0))
])

model_pipeline.fit(train_data['comment'], train_data['toxic'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x7f8e7ac46170>)),
                ('model', LogisticRegression(random_state=0))])

In [37]:
model_pipeline.predict(["Привет, что делаешь?"])

array([0])

In [38]:
model_pipeline.predict(["Ты убогий"])

array([1])

# 4. Evaluation of the model

To assess the quality of the algorithm on each of the classes separately, we introduce the metrics precision and recall.

![](https://hsto.org/getpro/habr/post_images/164/93b/c89/16493bc899f7275f3b5ff8d45a3ed2e2.svg)

Precision can be interpreted as the proportion of objects called positive by the classifier and at the same time really being positive, and recall shows what proportion of objects of a positive class out of all objects of a positive class the algorithm found.

![](https://sphire.mpg.de/wiki/lib/exe/fetch.php?cache=&media=pipeline:window:cryolo:precision_recall.png)

In [39]:
from sklearn.metrics import precision_score, recall_score, precision_recall_curve

print(f"Precision: {precision_score(test_data['toxic'], model_pipeline.predict(test_data['comment']))}")
print(f"Recall: {recall_score(test_data['toxic'], model_pipeline.predict(test_data['comment']))}")

Precision: 0.9196428571428571
Recall: 0.6688311688311688


In [56]:
from sklearn.metrics import precision_recall_curve

prec, rec, thresholds = precision_recall_curve(test_data['toxic'], model_pipeline.predict_proba(test_data['comment'])[:, 1])
np.where(prec > 0.95)

(array([417, 418, 453, 454, 455, 456, 457, 458, 471, 472, 473, 474, 475,
        476, 477, 478, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489,
        490, 491, 492, 493, 494, 495, 496, 497, 498]),)

In [57]:
print(f"Precision: {precision_score(test_data['toxic'], model_pipeline.predict_proba(test_data['comment'])[:, 1] > thresholds[345])}")
print(f"Recall: {recall_score(test_data['toxic'], model_pipeline.predict_proba(test_data['comment'])[:, 1] > thresholds[345])}")

Precision: 0.8366013071895425
Recall: 0.8311688311688312


# 5. Selection of optimal parameters for the model

In [41]:
from sklearn.model_selection import GridSearchCV


param_grid = {'C': [0.1, 1, 10.]}

grid_pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x, remove_stop_words = True))),
    ('model',  GridSearchCV(LogisticRegression(random_state = 0, max_iter=200), param_grid, cv=3, verbose=4))
])

grid_pipeline.fit(train_data['comment'], train_data['toxic'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV 1/3] END .............................C=0.1;, score=0.686 total time=   0.4s
[CV 2/3] END .............................C=0.1;, score=0.687 total time=   0.8s
[CV 3/3] END .............................C=0.1;, score=0.685 total time=   0.8s
[CV 1/3] END ...............................C=1;, score=0.829 total time=   1.4s
[CV 2/3] END ...............................C=1;, score=0.842 total time=   0.9s
[CV 3/3] END ...............................C=1;, score=0.837 total time=   1.4s
[CV 1/3] END ............................C=10.0;, score=0.854 total time=   3.1s
[CV 2/3] END ............................C=10.0;, score=0.865 total time=   2.4s
[CV 3/3] END ............................C=10.0;, score=0.872 total time=   1.9s


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x7f8e7a751480>)),
                ('model',
                 GridSearchCV(cv=3,
                              estimator=LogisticRegression(max_iter=200,
                                                           random_state=0),
                              param_grid={'C': [0.1, 1, 10.0]}, verbose=4))])

In [ ]:
model_pipeline_c_10 = Pipeline([
    ('vectorizer', TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x, remove_stop_words = True))),
    ('model', LogisticRegression(random_state = 0, C=10, max_iter=200))
])

model_pipeline_c_10.fit(train_data['comment'], train_data['toxic'])

In [ ]:
prec_c_10, rec_c_10, thresholds_c_10 = precision_recall_curve(test_data['toxic'], model_pipeline_c_10.predict_proba(test_data['comment'])[:, 1])

In [ ]:
np.where(prec_c_10 > 0.95)

In [ ]:
print(f"Precision: {precision_score(test_data['toxic'], model_pipeline_c_10.predict_proba(test_data['comment'])[:, 1] > thresholds_c_10[271])}")
print(f"Recall: {recall_score(test_data['toxic'], model_pipeline_c_10.predict_proba(test_data['comment'])[:, 1] > thresholds_c_10[271])}")

In [43]:
!pip3 install dill
import pickle
import dill
from sklearn.feature_extraction.text import CountVectorizer

# save
pkl_filename = "pickle_model.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(model, file)

vct_filename = "vectorizer.pkl" 
with open(vct_filename, 'wb') as file: 
    dill.dump(vectorizer, file)
    
# open
with open(pkl_filename, 'rb') as file: 
    pickle_model = pickle.load(file)
    
with open(vct_filename, 'rb') as file: 
    pickle_vectorizer = dill.load(file)  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.4 MB/s eta 0:00:00


In [54]:
import time

def check(text):
    start_time = time.time()
    answer = pickle_model.predict(
        pickle_vectorizer.transform([text])
    )

    return {
        "text": text,
        "answer": 'toxic' if answer[0] else 'neutral',
        "time": f'{time.time() - start_time}s'
    }

In [55]:
messages = ["ты тупой", "привет, как дела?"]

for message in messages:
  print(check(message))

{'text': 'ты тупой', 'answer': 'toxic', 'time': '0.002236604690551758s'}
{'text': 'привет, как дела?', 'answer': 'neutral', 'time': '0.0036804676055908203s'}
